In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import observesim.cadence as cadence

# Determining field and target cadences

Here we explore some tools to determine field and target cadences. We assume that the targets in the field are given to us with heterogeneous constraints on cadence encapsulated with the following information:

 * nepochs
 * epoch[] (desired epoch after first observation, days)
 * softness[] (tolerance on epoch, days)
 * lunation[] (maximize allowed lunation value)
 * value

We define lunation as the illumination fraction of the moon, or 0 if the Moon is below the horizon. 

If you want several exposures at a given epoch, you define multiple entries with the same epoch value.

## Cadence Consistency Matrix

We need some tools to decide on the field cadence. A basic question is:
"Is cadence $i$ achievable in a field with cadence $j$?"

For example, a single exposure cadence is always achievable, but if we
need two exposures on a target in cadence $i$ then its achievability
depends on the requirements on their cadence and what is on offer
under cadence $j$. That is, if cadence i needs a month separation, but
cadence $j$ has yearly separations, then it won't work. 

If we can determine this, we can construct a "cadence consistency
matrix", $C_{ij}$ which contains the answer to the above question
encoded as a 0 or 1 (i.e. False or True). Note that this matrix is
decidedly not symmetric. This matrix will be the basis on which we
make decisions about which cadences to go for. 

To determine the cadence consistency matrix, we have to look at the
individual exposures. Imagine we want to get exposures in the cadence
specified in cadence $i$. Well, the first exposure in the list needs to
correspond to some exposure in cadence $j$. So we will check each
possibility in turn.  

For each choice of epoch_i(0) = epoch_i(jstart), we now create an Exposure
Consistency Matrix. The exposure consistency matrix contains the
answer to the question for each $i'$, $j'$:

 "Is exposure $i'$ consistent with exposure $j'$ offset by epoch_j(jstart)?"

The Exposure Consistency Matrix, $E_{ij}$ is 0s and 1s. The condition
that the cadence i is achievable under cadence $j$ is equivalent to
being able to construct a matrix $w_{ij}$ for which the following is
true for some choice of jstart:

    w_{ij} <= E_{ij} for all i, j, AND 
    \sum_i w_{ij} <= 1 for all j, AND
    \sum_j w_{j} == 1 for all i

The $w_{ij}$ are the "solutions" -- the choices of which exposures for
cadence $i$ to put in which exposures for cadence $j$.

The Exposure Consistency Matrix and its solutions will be important in
selecting targets to assign to fibers for each design as well.

The solutions to the Exposure Consistency Matrix problem can be found
with standard techniques in constraint programming. Thus, we can
determine the Cadence Consistency Matrix. Here we use Google's ortools 
package.

## CadenceList objects

The CadenceList objects in observesim have tools for performing this operation. We start by creating a cadence list, which by default has nothing in it.

In [2]:
cadences = cadence.CadenceList()

Now let us add some basic cadences. In the epochs, note that we always start with "0", because it doesn't really matter when the cadence starts.

In [3]:
# One epoch, any lunation
cadences.add_cadence(nexposures=1, epoch=[0.], softness=[5.], lunation=[1.])
# Two epochs, separated by about 20 days 
cadences.add_cadence(nexposures=2, epoch=[0., 20.], softness=[5., 4.], lunation=[1., 1.])
# Three epochs
cadences.add_cadence(nexposures=3, epoch=[0., 20., 30.], softness=[5., 2., 2.], lunation=[1., 1., 1.])
# Four epochs
cadences.add_cadence(nexposures=4, epoch=[0., 20., 30., 50.], softness=[5., 2., 2., 2.], lunation=[1., 1., 1., 1.])

Now we can ask whether any cadence is achievable within any other one. We refer to the cadence by the zero-indexed order in which they are added to the list (we may want to clean this bad practice up!). 

For example, the one-epoch cadence is consistent within any of the others. If you ask for the solutions, you get a list of them; each solution is itself a list of epochs.

In [4]:
print(cadences.cadence_consistency(0, 1, return_solutions=True))
print(cadences.cadence_consistency(0, 2, return_solutions=True))
print(cadences.cadence_consistency(0, 3, return_solutions=True))

(2, [[0], [1]])
(3, [[0], [1], [2]])
(4, [[0], [1], [2], [3]])


But no other cadence is consistent within the one-epoch cadence:

In [5]:
print(cadences.cadence_consistency(1, 0, return_solutions=True))

(0, [])


In our case, the two epoch cadence is consistent within the four-epoch cadences, with two choices for which epochs to associate with it:

In [6]:
cadences.cadence_consistency(1, 3, return_solutions=True)

(2, [[0, 1], [2, 3]])

## Assigning targets to epochs

Now imagine that you have a set of targets, each with a desired cadence. For example, these may be the targets reachable by a particular robot arm for a particular choice of field. You need to pick a field cadence that can include those target cadences. Once you have done that you need to assign the targets to each epoch in such a way as to maximize some concept of their value.

This problem can be solved using constraint programming. Conceptually we construct a list of "allowed" choices indexed by target $i$, choice of cadence $j$, and epoch $k$. We set $A_{ijk}=1$ for each allowed choice, and $0$ for each not allowed choice. (This is not exactly a matrix, since the number of allowed cadences $N_{c,i}$ is different for each target $i$). 

We then need to choose for each $ijk$ whether or not there is an associated observation, which we indicate with $w_{ijk}$. 

The conditions to satisfy are:

    w_{ijk} <= A_{ijk}
    for all k, \sum_{ij} w_{ijk} <= 1 
    for all i, \sum_j [ (\sum_{k} w_{ijk}) > 0 ] <= 1
    for each i and j, w_{ijk} are all equal for k for which A_{ijk} = 1
    
Then, for some specified value for each target $v_i$, we want to find a solution that maximizes the total $\sum_i v_i$. 

These conditions can be specified and solved with ortools. This is implemented within the CadenceList class. The method pack_targets() takes a list of target cadences, a specified field cadence, and returns the targets to observe at each epoch.

For example, we can try to fit in four one-epoch targets into the 4-epoch cadence:

In [7]:
epoch_targets = cadences.pack_targets(target_cadences=[0, 0, 0, 0], field_cadence=3)
print(epoch_targets)

[3 2 1 0]


If we put five targets in, we will miss one:

In [8]:
epoch_targets = cadences.pack_targets(target_cadences=[0, 0, 0, 0, 0], field_cadence=3)
print(epoch_targets)

[4 3 2 1]


We can use the values if one target is less valuable than the others:

In [9]:
value = [2, 2, 1, 2, 2]
epoch_targets = cadences.pack_targets(target_cadences=[0, 0, 0, 0, 0], value=value, field_cadence=3)
print(epoch_targets)

[4 3 1 0]


We can do more complex cases too, to fit in two one-epoch targets and one two-epoch target:

In [10]:
epoch_targets = cadences.pack_targets(target_cadences=[0, 0, 1], field_cadence=3)
print(epoch_targets)

[2 2 1 0]
